In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql.functions import col, when


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/11/14 13:29:43 WARN Utils: Your hostname, ante-ThinkPad-T15-Gen-2i resolves to a loopback address: 127.0.1.1; using 192.168.6.93 instead (on interface wlp0s20f3)
23/11/14 13:29:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/14 13:29:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet

--2023-11-14 13:29:44--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.161.127.198, 3.161.127.221, 3.161.127.152, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.161.127.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-01.parquet.1’

fhv_tripdata_2021-0 100%[===================>]  11,33M  10,7MB/s    in 1,1s    

2023-11-14 13:29:45 (10,7 MB/s) - ‘fhv_tripdata_2021-01.parquet.1’ saved [11886281/11886281]



In [4]:
!wc -l fhv_tripdata_2021-01.parquet

32645 fhv_tripdata_2021-01.parquet


In [5]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhv_tripdata_2021-01.parquet')

df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropOff_datetime', TimestampNTZType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [6]:
pd.read_parquet('fhv_tripdata_2021-01.parquet').head(1000).to_parquet('head.parquet', index= False)

df_pandas = pd.read_parquet('head.parquet')
df_pandas['pickup_datetime'] = df_pandas['pickup_datetime'].apply(lambda x: None if pd.isna(x) else x)
df_pandas['dropOff_datetime'] = df_pandas['dropOff_datetime'].apply(lambda x: None if pd.isna(x) else x)




schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.DoubleType(), True),
    types.StructField('DOLocationID', types.DoubleType(), True),
    types.StructField('SR_Flag', types.IntegerType(), True)
])




In [8]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet('fhv_tripdata_2021-01.parquet')

In [9]:
df = df.repartition(24)
df = df.withColumn('SR_Flag', when(col('SR_Flag').isNull(), '').otherwise(col('SR_Flag').cast('string')))


In [10]:
df.select('SR_Flag').distinct().show()

+-------+
|SR_Flag|
+-------+
|       |
+-------+



In [11]:
df.write.parquet('fhvhv/2021/01/')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/ante/Desktop/de-z/week5/fhvhv/2021/01 already exists. Set mode as "overwrite" to overwrite the existing path.

In [38]:
df = spark.read.parquet('fhvhv/2021/01/')

In [15]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PULocationID: double (nullable = true)
 |-- DOLocationID: double (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [16]:
from pyspark.sql import functions as F

In [17]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|             null|              B02930|2021-01-11 09:14:37|2021-01-11 09:56:30|        null|        69.0|       |
|             null|              B01437|2021-01-21 15:06:18|2021-01-21 15:13:59|        null|        10.0|       |
|             null|              B02133|2021-01-29 14:17:03|2021-01-29 14:52:31|        null|        null|       |
|             null|              B01667|2021-01-03 15:54:37|2021-01-03 16:05:09|        null|        92.0|       |
|             null|              B01051|2021-01-24 13:34:46|2021-01-24 13:38:08|        null|       147.0|       |
|             null|              B00887|2021-01-24 15:40:35|2021-01-24 16:25:02|

In [18]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [19]:
crazy_stuff('B02884')

's/b44'

In [20]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [21]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropOff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b72| 2021-01-11|  2021-01-11|        null|        69.0|
|  a/59d| 2021-01-21|  2021-01-21|        null|        10.0|
|  a/855| 2021-01-29|  2021-01-29|        null|        null|
|  e/683| 2021-01-03|  2021-01-03|        null|        92.0|
|  e/41b| 2021-01-24|  2021-01-24|        null|       147.0|
|  e/377| 2021-01-24|  2021-01-24|        null|       265.0|
|  e/53b| 2021-01-29|  2021-01-29|        null|        42.0|
|  e/358| 2021-01-15|  2021-01-15|        null|        89.0|
|  e/53c| 2021-01-28|  2021-01-28|        null|       213.0|
|  s/b21| 2021-01-18|  2021-01-18|        null|        17.0|
|  a/bbe| 2021-01-25|  2021-01-25|       131.0|        70.0|
|  a/384| 2021-01-06|  2021-01-06|        null|       225.0|
|  a/384| 2021-01-15|  2021-01-15|        null|       180.0|
|  e/5a5| 2021-01-27|  2

In [39]:
df.select('pickup_datetime', 'dropOff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV003').show()

+---------------+----------------+------------+------------+
|pickup_datetime|dropOff_datetime|PULocationID|DOLocationID|
+---------------+----------------+------------+------------+
+---------------+----------------+------------+------------+



In [ ]:
!head -n 10 head.csv